In [1]:
import polars as pl
pl.Config(fmt_str_lengths=200)

In [2]:
import os
os.getcwd()

'/home/jovyan/smirnov'

In [3]:
test = pl.read_csv('/home/jovyan/smirnov/our_submissions/test_dataset_submission_queries.csv')

In [4]:
test.head(5)

query
str
"""Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия"""
"""битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть"""
"""Экстрасенсы. Битва сильнейших, 4 выпуск"""
"""супер стар 4 сезон"""
"""пять ночей с Фредди"""


In [5]:
import txtai
embeddings = txtai.Embeddings()
embeddings.load("/home/jovyan/smirnov/our_indexes/all_videos")

In [18]:
embeddings.search('битва сильнейших',10)

[{'id': 'video_3626279',
  'text': 'смотреть экстрасенсы битва сильнейших\nэкстрасенсы битва сильнейших\nэкстрасенсы битва сильнейших',
  'score': 0.4111642891194302},
 {'id': 'video_3048626',
  'text': 'экстрасенсы битва сильнейших/ битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023',
  'score': 0.4007101031847948},
 {'id': 'video_727356',
  'text': 'смотреть экстрасенсы битва сильнейших\nэкстрасенсы битва сильнейших\nэкстрасенсы битва сильнейших смот',
  'score': 0.4007101031847948},
 {'id': 'video_18943317',
  'text': 'битва сильнейших',
  'score': 0.3991782967750335},
 {'id': 'video_17799854',
  'text': 'Битва сильнейших',
  'score': 0.3991782967750335},
 {'id': 'video_19455333',
  'text': 'Битва сильнейших',
  'score': 0.3991782967750335},
 {'id': 'video_6906485',
  'text': 'битва сильнейших',
  'score': 0.3991782967750335},
 {'id': 'video_18542502',
  'text': 'Битва Сильнейших',
  'score': 0.3991782967750335},
 {'id': 'video_12043786',
  'text': 'Битва Сильней

In [31]:
q = 'Экстрасенсы. Битва сильнейших, 4 выпуск 2023'
limit=5
embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q}") \
order by score desc, v_pub_datetime desc \
limit {limit}')

[{'id': 'video_25673767',
  'text': 'Экстрасенсы. Битва сильнейших 4 выпуск (2023)',
  'v_pub_datetime': '2023-04-10 04:50:48',
  'score': 0.5570344481139637},
 {'id': 'video_3366872',
  'text': 'Экстрасенсы. Битва сильнейших, 4 выпуск',
  'v_pub_datetime': '2023-04-22 18:00:14',
  'score': 0.5270358005716969},
 {'id': 'video_5092245',
  'text': 'Экстрасенсы. Битва Сильнейших 2023 ВЫПУСК 1.04.2023',
  'v_pub_datetime': '2023-04-03 08:00:13',
  'score': 0.501986969676225},
 {'id': 'video_9287735',
  'text': 'Экстрасенсы. Битва Сильнейших 4 выпуск 22 апреля 2023',
  'v_pub_datetime': '2023-04-21 11:21:03',
  'score': 0.49888991489075984},
 {'id': 'video_17866921',
  'text': 'Экстрасенсы. Битва сильнейших 22 выпуск 4 ноября 2023',
  'v_pub_datetime': None,
  'score': 0.49888991489075984}]

In [75]:
import re
submission={"query" :[], "video_id":[]}
for q in test['query']:
    #print(q)
    q_clean=re.sub("[^а-яА-ЯЁё0-9a-zA-Z ]","",q)
    #print(q)
    limit=5
    res = embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q_clean}") \
    order by score desc, v_pub_datetime desc \
    limit {limit}')
    submission['query'] +=[q]*len(res)
    submission['video_id'] +=(list(map(lambda x: x['id'],res)))
    #print(res)

2

In [19]:
import nltk 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('popular')
# Очистка текста (пример функции)
def clean_text(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    tokens = nltk.word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stopwords.words('russian')]
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

In [20]:
import re
submission = {"query": [], "video_id": []}
limit = 5  # Fixed number of results per query
for q in test['query']:
    q_clean = clean_text(re.sub("[^а-яА-ЯЁё0-9a-zA-Z ]", "", q))

    # Execute search
    res = embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q_clean}") \
    order by score desc, v_pub_datetime desc \
    limit {limit}')

    # If there are results, extend the submission dictionary with these results
    submission['query'] += [q] * 5
    submission['video_id'] += [x['id'] for x in res]

    # If the results are fewer than the limit, fill the remaining slots with empty values
    if len(res) < limit:
        submission['video_id'] += [None] * (limit - len(res))

In [21]:
len(submission['query'])

10000

In [9]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 2000)

In [10]:
sub_df = pd.DataFrame(submission)
sub_df.head(10)

,query,video_id
0,"Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия",video_13417135
1,"Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия",video_3048626
2,"Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия",video_3626279
3,"Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия",video_24627118
4,"Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия",video_12827624
5,битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_33212734
6,битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_29499323
7,битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_9558708
8,битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_11083436
9,битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_7205638


In [22]:
sub_df.to_csv('/home/jovyan/smirnov/our_submissions/txtai_all_submission_04_stop_words.csv',index=False)